# Úkol č. 4 - regrese (do 14. ledna, nejpozději však před zkouškou)

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných (ale celkem vyčištěných) datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdroj dat je zde: https://data.world/uci/online-news-popularity.
  * Popis datasetu najdete v souboru `data_description.txt`.
  

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Proveďte základní průzkum dat a příp. vyhoďte nezajímavé příznaky.
  * Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `median_absolute_error`, bezpracné použití lineární regrese dává chybu zhruba 1700, Vaším úkolem je toto zlepšit.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Použijte i jiné metody než je lineární a hřebenová regrese.
  * (až +4 body) Získejte opravdu dobré výsledky (ve srovnání s Vašimi kolegy).

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte pouze tento Jupyter Notebook, opravujíví by neměl nic jiného potřebovat.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [1]:
import pandas as pd
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from scipy.optimize import minimize_scalar
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import ParameterGrid
import numpy as np
np.warnings.filterwarnings('ignore')
from sklearn.preprocessing import scale
from sklearn.preprocessing import robust_scale
from sklearn.preprocessing import normalize
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import power_transform
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
def split_data(data):
    return train_test_split(data.drop(columns = ['shares']), data['shares'], test_size=0.25, random_state = 44)

In [3]:
# v dokumentaci se pise, ze priznaky url a timedelta neovlivnuji shares, tak je mazu
# dale delim data na trenovaci a na konecne testovaci
df = pd.read_csv('data.csv')
df.columns = [col.strip() for col in df.columns]

df = df.drop(['url','timedelta'],axis=1)
X_first_train, X_final_test, Y_first_train, Y_final_test = split_data(df)
X_first = pd.concat([X_first_train,Y_first_train],axis=1)
X_first = X_first.reset_index(drop=True)

In [4]:
# vytvorim si list s linearni zavislosti na priznak shares
def linear_corr_train(data):
    a = data.corr().shares
    a = abs(a)
    a = a.sort_values(ascending=False)
    a.drop(a.head(1).index,inplace=True)
    return a

# ruzne zmeny dat

# vytvarim nove priznaky nasobenim jiz existujicich (podle list zavislosti) a pak mazu ty na spodu listu 
def data_change_1_do(data,names):
    res = data.copy()
    j = names.size
    for i in range(21):
        j = j - 1
        res[i] = res[names.index[j]] * res[names.index[i]]
    res =  res.drop(names.tail(20).index,axis=1)   
    return res

# vytvarim nove priznaky nasobenim priznaku (vedle sebe v listu zavislosti)
def data_change_2_do(data,names):
    res = data.copy()
    for i in range(0,names.size,2):
        res[i] = res[names.index[i]] * res[names.index[i+1]]
    return res


# mazu priznaky na spodu listu
def data_change_3_do(data,names):
    res = data.drop(names.tail(20).index,axis=1)
    return res

# nic nedelam
def data_change_4_do(data,names):
    return data

In [5]:
# ruzna skalovani dat

def quantil(data):
    return quantile_transform(data)
def power(data):
    return power_transform(data, method ='yeo-johnson')
def robust(data):
    return robust_scale(data)
def standard_scale(data):
    return scale(data)
def normal(data):
    return normalize(data)
def nothing(data):
    return data

In [16]:
# dale se nachazeji ruzne regresni modely
# kvuli rychlosti jsem u volby hyperparametru omezil parametry jenom na jednu nebo par moznosti

def neural(train_X,train_Y,test_X,test_Y):
    param_grid = {
        'hidden_layer_sizes': [(50,)],
        'random_state':  range(3,4,1)
    }
    param_comb = ParameterGrid(param_grid)
    
    def get_neural_clf(X_cvs,Y_cvs):
        
        def neuralmodel(param):
            clf = MLPRegressor(**param)
            return -np.mean(cross_val_score(clf,X_cvs,Y_cvs,scoring='neg_median_absolute_error',cv=3))
                
        params_res = []
        for param in param_comb:
            params_res.append(neuralmodel(param))
        return param_comb[params_res.index(np.min(params_res))]
    
    params_res = get_neural_clf(train_X,train_Y)
    reg = MLPRegressor(**params_res).fit(train_X,train_Y)
    return predict(reg,test_X,test_Y),reg

In [7]:
def AdaBoost(train_X,train_Y,test_X,test_Y):
    param_grid = {
        'n_estimators': range(30,40,20),
        'learning_rate': [0.05 ]
    }
    param_comb = ParameterGrid(param_grid)
    
    def get_forest_clf(X_cvs,Y_cvs):
        
        def forestmodel(param):
            clf = AdaBoostRegressor(**param)
            return -np.mean(cross_val_score(clf,X_cvs,Y_cvs,scoring='neg_median_absolute_error',cv=3))
                
        params_res = []
        for param in param_comb:
            params_res.append(forestmodel(param))
        return param_comb[params_res.index(np.min(params_res))]
    
    params_res = get_forest_clf(train_X,train_Y)
    reg = AdaBoostRegressor(**params_res).fit(train_X,train_Y)
    return predict(reg,test_X,test_Y),reg

In [8]:
def randomForest(train_X,train_Y,test_X,test_Y):
    param_grid = {
        'n_estimators': range(30,40,20),
        'max_depth': range(2,3)
    }
    param_comb = ParameterGrid(param_grid)
    
    def get_forest_clf(X_cvs,Y_cvs):
        
        def forestmodel(param):
            clf = RandomForestRegressor(**param)
            return -np.mean(cross_val_score(clf,X_cvs,Y_cvs,scoring='neg_median_absolute_error',cv=3))
        
        params_res = []
        for param in param_comb:
            params_res.append(forestmodel(param))
        return param_comb[params_res.index(np.min(params_res))]
    params_res = get_forest_clf(train_X,train_Y)
    reg = RandomForestRegressor(**params_res).fit(train_X,train_Y)
    return predict(reg,test_X,test_Y),reg

In [9]:
def linear(train_X,train_Y,test_X,test_Y):
    reg = LinearRegression().fit(train_X, train_Y)
    return predict(reg,test_X,test_Y),reg

In [10]:
def hreben(train_X,train_Y,test_X,test_Y):
    param_grid = {
        'alpha': np.arange(1.0, 10.0, 0.5)
    }
    param_comb = ParameterGrid(param_grid)
    
    
    def get_ridge_clf(X_cvs,Y_cvs):
        
        def ridgemodel(param):
            clf = Ridge(**param)
            return -np.mean(cross_val_score(clf,X_cvs,Y_cvs,scoring='neg_median_absolute_error',cv=3))
        
        params_res = []
        for param in param_comb:
            params_res.append(ridgemodel(param))
        return param_comb[params_res.index(np.min(params_res))]
    
    params_res = get_ridge_clf(train_X,train_Y)
    reg = Ridge(**params_res).fit(train_X, train_Y)
    return predict(reg,test_X,test_Y),reg    

In [11]:
def predict(reg,test_X,test_Y):
    predict = reg.predict(test_X)
    return median_absolute_error(test_Y,predict)

In [12]:
# kombinuji vsechny moje moznosti upravy dat a modely
# vsechny vysledky uschovavam do listu vys

# kvuli rychlosti jsou momentalne spustene jen linearni a hrebenove regrese s upravou dat, u kterych davaly nejlepsi vysledek

data_changes = [data_change_2_do]
#data_changes = [data_change_1_do,data_change_2_do,data_change_3_do,data_change_4_do]

lin_corr_list = linear_corr_train(X_first)

scales = [standard_scale]
#scales = [quantil,power,robust,standard_scale,normal,nothing]

vys = []

for cur_scale in scales:
    data = pd.DataFrame(cur_scale(X_first.drop(columns = ['shares'])),columns = X_first.drop(columns = ['shares']).columns)
    data = pd.concat([data, X_first['shares']], axis=1)
    for cur_change in data_changes:
        data1 = cur_change(data,lin_corr_list)
        X,X_val,Y,Y_val = split_data(data1)
        tup = hreben(X,Y,X_val,Y_val)                          
        vys.append((tup[0],tup[1],cur_scale,cur_change))
        tup = linear(X,Y,X_val,Y_val)                          
        vys.append((tup[0],tup[1],cur_scale,cur_change))
        #tup = randomForest(X,Y,X_val,Y_val)                     
        #vys.append((tup[0],tup[1],cur_scale,cur_change))
        #tup = AdaBoost(X,Y,X_val,Y_val)                        
        #vys.append((tup[0],tup[1],cur_scale,cur_change))
        #tup = neural(X,Y,X_val,Y_val)                            
        #vys.append((tup[0],tup[1],cur_scale,cur_change))

In [13]:
# vypis nejlepsi moznosti s chybou 'median_absolute_error' s testovacimi daty 
# (pouze linearni a hrebenova regrese)

limit = vys[0][0]
final = vys[0]
for i in vys:
    if (i[0] < limit):
        final = i
        limit = i[0]
final_error = predict(final[1],final[2](final[3](X_final_test,lin_corr_list)),Y_final_test)
print('Konecna chyba je ',final_error,'\ns modelem ',final[1],',\n pouzite funkce na skalovani dat : ',final[2],'\na funkce na zmenu priznaku: ',final[3])

Konecna chyba je  1590.9466082487897 
s modelem  Ridge(alpha=5.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001) ,
 pouzite funkce na skalovani dat :  <function standard_scale at 0x7f47881dcb70> 
a funkce na zmenu priznaku:  <function data_change_2_do at 0x7f478bed6d90>


In [17]:
# vypocet vsech modelu s upravou dat, u kterych davaji nejlepsi vysledek

lin_corr_list = linear_corr_train(X_first)
vys1 = []

data = pd.DataFrame(standard_scale(X_first.drop(columns = ['shares'])),columns = X_first.drop(columns = ['shares']).columns)
data = pd.concat([data, X_first['shares']], axis=1)
data1 = data_change_2_do(data,lin_corr_list)
X,X_val,Y,Y_val = split_data(data1)
tup = hreben(X,Y,X_val,Y_val)                       
vys1.append((tup[0],tup[1],standard_scale,data_change_2_do))


data = pd.DataFrame(standard_scale(X_first.drop(columns = ['shares'])),columns = X_first.drop(columns = ['shares']).columns)
data = pd.concat([data, X_first['shares']], axis=1)
data1 = data_change_2_do(data,lin_corr_list)
X,X_val,Y,Y_val = split_data(data1)
tup = linear(X,Y,X_val,Y_val)                           
vys1.append((tup[0],tup[1],standard_scale,data_change_2_do))


data = pd.DataFrame(nothing(X_first.drop(columns = ['shares'])),columns = X_first.drop(columns = ['shares']).columns)
data = pd.concat([data, X_first['shares']], axis=1)
data1 = data_change_3_do(data,lin_corr_list)
X,X_val,Y,Y_val = split_data(data1)
tup = randomForest(X,Y,X_val,Y_val)                     
vys1.append((tup[0],tup[1],nothing,data_change_3_do))


data = pd.DataFrame(power(X_first.drop(columns = ['shares'])),columns = X_first.drop(columns = ['shares']).columns)
data = pd.concat([data, X_first['shares']], axis=1)
data1 = data_change_3_do(data,lin_corr_list)
X,X_val,Y,Y_val = split_data(data1)
tup = AdaBoost(X,Y,X_val,Y_val)                         
vys1.append((tup[0],tup[1],power,data_change_3_do))


data = pd.DataFrame(robust(X_first.drop(columns = ['shares'])),columns = X_first.drop(columns = ['shares']).columns)
data = pd.concat([data, X_first['shares']], axis=1)
data1 = data_change_2_do(data,lin_corr_list)
X,X_val,Y,Y_val = split_data(data1)
tup = neural(X,Y,X_val,Y_val)                            
vys1.append((tup[0],tup[1],standard_scale,data_change_2_do))

In [18]:
# nejlepsi moznost ze vsech modelu
# vypis chyby nad testovacimi daty

limit = vys1[0][0]
final = vys1[0]
for i in vys1:
    if (i[0] < limit):
        final = i
        limit = i[0]
final_error1 = predict(final[1],final[2](final[3](X_final_test,lin_corr_list)),Y_final_test)
print('Konecna chyba je ',final_error1,'\ns modelem ',final[1],',\n pouzite funkce na skalovani dat : ',final[2],'\na funkce na zmenu priznaku: ',final[3])

Konecna chyba je  1289.9248542037958 
s modelem  MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=3, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False) ,
 pouzite funkce na skalovani dat :  <function standard_scale at 0x7f47881dcb70> 
a funkce na zmenu priznaku:  <function data_change_2_do at 0x7f478bed6d90>
